# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [43]:
%%capture
#RUN FIRST, installs a missing library
import sys
!{sys.executable} -m pip install lxml==4.4.1

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    nhl_df=pd.read_csv("assets/nhl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    nhl_df["team"] = nhl_df["team"].apply(lambda x:re.split('\*',x)[0])
    nhl_df=nhl_df[nhl_df["year"]==2018][["team","W","L"]].where(nhl_df["W"].apply(lambda x:x.isnumeric())).dropna()
    nhl_df["win_to_loss"]=nhl_df["W"].astype(int)/(nhl_df["L"].astype(int)+nhl_df["W"].astype(int))
    nhl_df["city"]=nhl_df["team"].apply(lambda x:re.split(' ',x)[0])
    city_map = {"Tampa":"Tampa Bay Area",
                "Florida":"Miami–Fort Lauderdale",
                "New":"New York City",
                "Washington":"Washington, D.C.",
                "Carolina":"Raleigh",
                "Minnesota":"Minneapolis–Saint Paul",
                "Colorado":"Denver",
                "St.":"St. Louis",
                "Dallas":"Dallas–Fort Worth",
                "Vegas":"Las Vegas",
                "Anaheim":"Los Angeles",
                "San":"San Francisco Bay Area",
                "Los":"Los Angeles",
                "Arizona":"Phoenix"}
    nhl_df["city"] = nhl_df["city"].map(city_map).fillna(nhl_df["city"])
    nhl_df = nhl_df.groupby(["city"]).agg({"win_to_loss":np.mean})
    merge_nhl = nhl_df.merge(cities,how="left",left_index=True,right_on="Metropolitan area")[["win_to_loss","Metropolitan area","Population (2016 est.)[8]"]]
    merge_nhl["Population (2016 est.)[8]"] = merge_nhl["Population (2016 est.)[8]"].astype(np.int64)
    
    population_by_region = merge_nhl["Population (2016 est.)[8]"].tolist() # pass in metropolitan area population from cities
    win_loss_by_region = merge_nhl["win_to_loss"].tolist() # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    stats.pearsonr(population_by_region, win_loss_by_region)    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [96]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    
    nba_df=pd.read_csv("assets/nba.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    nba_df["team"] = nba_df["team"].apply(lambda x:re.split('\*|\(',x)[0])
    nba_df = nba_df[nba_df["year"]==2018][["team","W/L%"]]
    nba_df["city"]=nba_df["team"].apply(lambda x:re.split(' ',x)[0])
    nba_df.loc[20,"city"] = "New Orleans"
    city_map = {"Tampa":"Tampa Bay Area",
            "Florida":"Miami–Fort Lauderdale",
            "New":"New York City",
            "Washington":"Washington, D.C.",
            "Carolina":"Raleigh",
            "Minnesota":"Minneapolis–Saint Paul",
            "Colorado":"Denver",
            "St.":"St. Louis",
            "Dallas":"Dallas–Fort Worth",
            "Vegas":"Las Vegas",
            "Anaheim":"Los Angeles",
            "San":"San Antonio",
            "Los":"Los Angeles",
            "Arizona":"Phoenix",
            "Indiana":"Indianapolis",
            "Miami":"Miami–Fort Lauderdale",
            "Brooklyn":"New York City",
            "Golden":"San Francisco Bay Area",
            "Oklahoma":"Oklahoma City",
            "Utah":"Salt Lake City",
                }
    nba_df["city"] = nba_df["city"].map(city_map).fillna(nba_df["city"])
    nba_df["W/L%"] = nba_df["W/L%"].astype(float)
    nba_df = nba_df.groupby(["city"]).agg({"W/L%":np.mean})
    merge_nba = nba_df.merge(cities,how="left",left_index=True,right_on="Metropolitan area")[["W/L%","Metropolitan area","Population (2016 est.)[8]"]]
    merge_nba["Population (2016 est.)[8]"] = merge_nba["Population (2016 est.)[8]"].astype(np.int64)
    population_by_region = merge_nba["Population (2016 est.)[8]"].tolist() # pass in metropolitan area population from cities
    win_loss_by_region = merge_nba["W/L%"].tolist() # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    


    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [98]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    mlb_df=pd.read_csv("assets/mlb.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    mlb_df["team"] = mlb_df["team"].apply(lambda x:re.split('\*|\(',x)[0])
    mlb_df = mlb_df[mlb_df["year"]==2018][["team","W-L%"]]
    mlb_df["city"]=mlb_df["team"].apply(lambda x:re.split(' ',x)[0])
    mlb_df.loc[29,"city"] = "San Diego"
    city_map = {"Tampa":"Tampa Bay Area",
            "Florida":"Miami–Fort Lauderdale",
            "New":"New York City",
            "Washington":"Washington, D.C.",
            "Carolina":"Raleigh",
            "Minnesota":"Minneapolis–Saint Paul",
            "Colorado":"Denver",
            "St.":"St. Louis",
            "Dallas":"Dallas–Fort Worth",
            "Vegas":"Las Vegas",
            "Anaheim":"Los Angeles",
            "San":"San Francisco Bay Area",
            "Los":"Los Angeles",
            "Arizona":"Phoenix",
            "Indiana":"Indianapolis",
            "Miami":"Miami–Fort Lauderdale",
            "Brooklyn":"New York City",
            "Golden":"San Francisco Bay Area",
            "Oklahoma":"Oklahoma City",
            "Utah":"Salt Lake City",
            "Texas":"Dallas–Fort Worth",
            "Kansas":"Kansas City",
            "Oakland":"San Francisco Bay Area",
            }
    mlb_df["city"] = mlb_df["city"].map(city_map).fillna(mlb_df["city"])
    mlb_df["W-L%"] = mlb_df["W-L%"].astype(float)
    mlb_df = mlb_df.groupby(["city"]).agg({"W-L%":np.mean})
    merge_mlb = mlb_df.merge(cities,how="left",left_index=True,right_on="Metropolitan area")[["W-L%","Metropolitan area","Population (2016 est.)[8]"]]
    merge_mlb["Population (2016 est.)[8]"] = merge_mlb["Population (2016 est.)[8]"].astype(np.int64)
    population_by_region = merge_mlb["Population (2016 est.)[8]"].tolist() # pass in metropolitan area population from cities
    win_loss_by_region = merge_mlb["W-L%"].tolist() # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]
    

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [100]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    nfl_df["team"] = nfl_df["team"].apply(lambda x:re.split('\*|\+',x)[0])
    nfl_df=nfl_df[nfl_df["year"]==2018][["team","W","L"]].where(nfl_df["W"].apply(lambda x:x.isnumeric())).dropna()
    nfl_df["win_to_loss"]=nfl_df["W"].astype(int)/(nfl_df["L"].astype(int)+nfl_df["W"].astype(int))
    nfl_df["city"]=nfl_df["team"].apply(lambda x:re.split(' ',x)[0])
    nfl_df.loc[1,'city'] = 'Boston'
    nfl_df.loc[31,'city'] = 'New Orleans'
    city_map = {"Tampa":"Tampa Bay Area",
            "Florida":"Miami–Fort Lauderdale",
            "New":"New York City",
            "Washington":"Washington, D.C.",
            "Carolina":"Charlotte",
            "Minnesota":"Minneapolis–Saint Paul",
            "Colorado":"Denver",
            "St.":"St. Louis",
            "Dallas":"Dallas–Fort Worth",
            "Vegas":"Las Vegas",
            "Anaheim":"Los Angeles",
            "San":"San Francisco Bay Area",
            "Los":"Los Angeles",
            "Arizona":"Phoenix",
            "Indiana":"Indianapolis",
            "Miami":"Miami–Fort Lauderdale",
            "Brooklyn":"New York City",
            "Golden":"San Francisco Bay Area",
            "Oklahoma":"Oklahoma City",
            "Utah":"Salt Lake City",
            "Texas":"Dallas–Fort Worth",
            "Kansas":"Kansas City",
            "Oakland":"San Francisco Bay Area",
            "Green":"Green Bay",
            "Tennessee":"Nashville"}
    nfl_df["city"] = nfl_df["city"].map(city_map).fillna(nfl_df["city"])
    nfl_df = nfl_df.groupby(["city"]).agg({"win_to_loss":np.mean})
    merge_nfl = nfl_df.merge(cities,how="left",left_index=True,right_on="Metropolitan area")[["win_to_loss","Metropolitan area","Population (2016 est.)[8]"]]
    merge_nfl["Population (2016 est.)[8]"] = merge_nfl["Population (2016 est.)[8]"].astype(np.int64)
    population_by_region = merge_nfl["Population (2016 est.)[8]"].tolist() # pass in metropolitan area population from cities
    win_loss_by_region = merge_nfl["win_to_loss"].tolist() # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]


## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [92]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def sports_team_performance():
    mlb_df=pd.read_csv("assets/mlb.csv")
    nhl_df=pd.read_csv("assets/nhl.csv")
    nba_df=pd.read_csv("assets/nba.csv")
    nfl_df=pd.read_csv("assets/nfl.csv")
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    for sport in sports:
        cities[sport] = cities[sport].apply(lambda x:re.split("\[",str(x))[0])
    #NHL
    nhl_df["team"] = nhl_df["team"].apply(lambda x:re.split('\*',x)[0])
    nhl_df=nhl_df[nhl_df["year"]==2018][["team","W","L"]].where(nhl_df["W"].apply(lambda x:x.isnumeric())).dropna()
    nhl_df["W-L%(NHL)"]=nhl_df["W"].astype(int)/(nhl_df["L"].astype(int)+nhl_df["W"].astype(int))
    nhl_df["city"]=nhl_df["team"].apply(lambda x:re.split(' ',x)[0])
    city_map = {"Tampa":"Tampa Bay Area",
                "Florida":"Miami–Fort Lauderdale",
                "New":"New York City",
                "Washington":"Washington, D.C.",
                "Carolina":"Raleigh",
                "Minnesota":"Minneapolis–Saint Paul",
                "Colorado":"Denver",
                "St.":"St. Louis",
                "Dallas":"Dallas–Fort Worth",
                "Vegas":"Las Vegas",
                "Anaheim":"Los Angeles",
                "San":"San Francisco Bay Area",
                "Los":"Los Angeles",
                "Arizona":"Phoenix"}
    nhl_df["city"] = nhl_df["city"].map(city_map).fillna(nhl_df["city"])
    nhl_df = nhl_df.groupby(["city"]).agg({"W-L%(NHL)":np.mean})

    #NBA

    nba_df["team"] = nba_df["team"].apply(lambda x:re.split('\*|\(',x)[0])
    nba_df = nba_df[nba_df["year"]==2018][["team","W/L%"]]
    nba_df["city"]=nba_df["team"].apply(lambda x:re.split(' ',x)[0])
    nba_df.loc[20,"city"] = "New Orleans"
    city_map = {"Tampa":"Tampa Bay Area",
            "Florida":"Miami–Fort Lauderdale",
            "New":"New York City",
            "Washington":"Washington, D.C.",
            "Carolina":"Raleigh",
            "Minnesota":"Minneapolis–Saint Paul",
            "Colorado":"Denver",
            "St.":"St. Louis",
            "Dallas":"Dallas–Fort Worth",
            "Vegas":"Las Vegas",
            "Anaheim":"Los Angeles",
            "San":"San Antonio",
            "Los":"Los Angeles",
            "Arizona":"Phoenix",
            "Indiana":"Indianapolis",
            "Miami":"Miami–Fort Lauderdale",
            "Brooklyn":"New York City",
            "Golden":"San Francisco Bay Area",
            "Oklahoma":"Oklahoma City",
            "Utah":"Salt Lake City",
                }
    nba_df["city"] = nba_df["city"].map(city_map).fillna(nba_df["city"])
    nba_df["W/L%"] = nba_df["W/L%"].astype(float)
    nba_df = nba_df.groupby(["city"]).agg({"W/L%":np.mean})

    #MLB

    mlb_df["team"] = mlb_df["team"].apply(lambda x:re.split('\*|\(',x)[0])
    mlb_df = mlb_df[mlb_df["year"]==2018][["team","W-L%"]]
    mlb_df["city"]=mlb_df["team"].apply(lambda x:re.split(' ',x)[0])
    mlb_df.loc[29,"city"] = "San Diego"
    city_map = {"Tampa":"Tampa Bay Area",
            "Florida":"Miami–Fort Lauderdale",
            "New":"New York City",
            "Washington":"Washington, D.C.",
            "Carolina":"Raleigh",
            "Minnesota":"Minneapolis–Saint Paul",
            "Colorado":"Denver",
            "St.":"St. Louis",
            "Dallas":"Dallas–Fort Worth",
            "Vegas":"Las Vegas",
            "Anaheim":"Los Angeles",
            "San":"San Francisco Bay Area",
            "Los":"Los Angeles",
            "Arizona":"Phoenix",
            "Indiana":"Indianapolis",
            "Miami":"Miami–Fort Lauderdale",
            "Brooklyn":"New York City",
            "Golden":"San Francisco Bay Area",
            "Oklahoma":"Oklahoma City",
            "Utah":"Salt Lake City",
            "Texas":"Dallas–Fort Worth",
            "Kansas":"Kansas City",
            "Oakland":"San Francisco Bay Area",
            }
    mlb_df["city"] = mlb_df["city"].map(city_map).fillna(mlb_df["city"])
    mlb_df["W-L%"] = mlb_df["W-L%"].astype(float)
    mlb_df = mlb_df.groupby(["city"]).agg({"W-L%":np.mean})

    #NFL
    nfl_df["team"] = nfl_df["team"].apply(lambda x:re.split('\*|\+',x)[0])
    nfl_df=nfl_df[nfl_df["year"]==2018][["team","W","L"]].where(nfl_df["W"].apply(lambda x:x.isnumeric())).dropna()
    nfl_df["win_to_loss"]=nfl_df["W"].astype(int)/(nfl_df["L"].astype(int)+nfl_df["W"].astype(int))
    nfl_df["city"]=nfl_df["team"].apply(lambda x:re.split(' ',x)[0])
    nfl_df.loc[1,'city'] = 'Boston'
    nfl_df.loc[31,'city'] = 'New Orleans'
    city_map = {"Tampa":"Tampa Bay Area",
            "Florida":"Miami–Fort Lauderdale",
            "New":"New York City",
            "Washington":"Washington, D.C.",
            "Carolina":"Charlotte",
            "Minnesota":"Minneapolis–Saint Paul",
            "Colorado":"Denver",
            "St.":"St. Louis",
            "Dallas":"Dallas–Fort Worth",
            "Vegas":"Las Vegas",
            "Anaheim":"Los Angeles",
            "San":"San Francisco Bay Area",
            "Los":"Los Angeles",
            "Arizona":"Phoenix",
            "Indiana":"Indianapolis",
            "Miami":"Miami–Fort Lauderdale",
            "Brooklyn":"New York City",
            "Golden":"San Francisco Bay Area",
            "Oklahoma":"Oklahoma City",
            "Utah":"Salt Lake City",
            "Texas":"Dallas–Fort Worth",
            "Kansas":"Kansas City",
            "Oakland":"San Francisco Bay Area",
            "Green":"Green Bay",
            "Tennessee":"Nashville"}
    nfl_df["city"] = nfl_df["city"].map(city_map).fillna(nfl_df["city"])
    nfl_df = nfl_df.groupby(["city"]).agg({"win_to_loss":np.mean})

    #merge all

    merge_table = (nhl_df.merge(nba_df,how="outer",left_index=True,right_index=True)
                         .merge(mlb_df,how="outer",left_index=True,right_index=True)
                         .merge(nfl_df,how="outer",left_index=True,right_index=True))
    merge_table = merge_table.rename(columns={"W-L%(NHL)":"NHL","W/L%":"NBA","W-L%":"MLB","win_to_loss":"NFL"})
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)

    for sport1 in sports:
        for sport2 in sports:
            if sport1==sport2:
                pass
            else:
                data = merge_table[[sport1,sport2]].dropna()
                p_values.loc[sport1,sport2] = stats.ttest_rel(data[sport1],data[sport2]).pvalue
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values